# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from CheXpert.disease_prediction.dataset import CheXpertDiseaseDataset
from shared_utils import vprint, to_gpu, add_mean_to_list, Mode, SharedConfigs
import shared_utils
from CheXpert.disease_prediction.utils import Configs as DiseaseConfigs
from CheXpert.race_prediction.utils import Configs as RaceConfigs
from MIMIC_CXR.dataset import CXRDataset
from MIMIC_CXR.utils import Configs as CXRConfigs

# Configs

In [10]:
@dataclass
class Configs(SharedConfigs):
    CXR_DATA_DIR = os.path.join("data", "MIMIC-CXR-JPG")
    CXR_VALID_LABELS_FILENAME = "valid.csv"
    CXR_FILENAMES = CXRConfigs.CXR_FILENAMES
    CHEXPERT_DATA_DIR = os.path.join("data", "CheXpert", "CheXpert-v1.0-small")
    CHEXPERT_DISEASE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "disease_prediction", "trained_models")
    CHEXPERT_RACE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "race_prediction", "trained_models")
    CHEXPERT_VALID_LABELS_FILENAME = "valid.csv"
    CHEXPERT_DEMO_FILENAME = "CHEXPERT DEMO.csv"
    DISEASE_ANNOTATIONS_COLUMNS = DiseaseConfigs.ANNOTATIONS_COLUMNS
    RACE_ANNOTATIONS_COLUMNS = RaceConfigs.ANNOTATIONS_COLUMNS
    NUM_DISEASE_CLASSES = DiseaseConfigs.NUM_CLASSES
    NUM_RACE_CLASSES = RaceConfigs.NUM_CLASSES
    CHEXPERT_RACE_DICT = RaceConfigs.RACE_DICT
    BATCH_SIZE = 4
    VALID_SIZE_DEBUG = 10**5

In [11]:
shared_utils.set_seed(Configs.SEED)

In [12]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", Configs)
else: 
    vprint(f"No GPU Memory.", Configs)

2022-07-24 17:20: Memory info: 8.5 GB free GPU.


In [13]:
valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Disease Prediction 

## Validaiton Dataloaders

In [10]:
cxp_disease_valid_dataset = CheXpertDiseaseDataset(data_dir=Configs.CHEXPERT_DATA_DIR, 
                                                   labels_filename=Configs.CHEXPERT_VALID_LABELS_FILENAME,
                                                   transform=valid_transform)
cxp_disease_valid_dataset.df_labels = cxp_disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxp_disease_valid_dataloader = DataLoader(cxp_disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxp_disease_valid_dataset)

234

In [ ]:
# group_sample_size = {
#     "Black_40-70_F": 100,
#     "Hispanic_40-70_F": 100
# }

In [35]:
cxr_disease_valid_dataset = CXRDataset.download_dataset(500, Mode.Disease, Configs.CXR_DATA_DIR,
                                                        Configs.CXR_VALID_LABELS_FILENAME, **Configs.CXR_FILENAMES,
                                                        transform=valid_transform, target_transform=None)
# cxr_disease_valid_dataset = CXRDataset(Mode.Disease, Configs.CXR_DATA_DIR, Configs.CXR_VALID_LABELS_FILENAME,
#                                        transform=valid_transform)
cxr_disease_valid_dataset.df_labels = cxr_disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxr_disease_valid_dataloader = DataLoader(cxr_disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxr_disease_valid_dataset)

race      age    gender
Asian     20-40  F           851
                 M           649
          40-70  F          2252
                 M          3014
          70-90  F          1939
                 M          2159
Black     20-40  F          5262
                 M          3326
          40-70  F         19265
                 M         13393
          70-90  F          8872
                 M          4877
Hispanic  20-40  F          1865
                 M          2033
          40-70  F          5497
                 M          5351
          70-90  F          2105
                 M          1337
Other     20-40  F          1413
                 M          2468
          40-70  F          3873
                 M          6442
          70-90  F          3019
                 M          3540
White     20-40  F          9334
                 M          9295
          40-70  F         46113
                 M         65679
          70-90  F         40784
                 M 

NotImplementedError: 

## Pretrained Models 

In [12]:
_, _, files = next(os.walk(Configs.CHEXPERT_DISEASE_TRAINED_MODELS_DIR))
disease_trained_models = [os.path.join(Configs.CHEXPERT_DISEASE_TRAINED_MODELS_DIR, file) for file in files]
len(disease_trained_models)

1

In [15]:
disease_model = densenet121()
num_features = disease_model.classifier.in_features
disease_model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=Configs.NUM_DISEASE_CLASSES, bias=True)
)
disease_model.eval()
not disease_model.training

True

In [16]:
disease_model, results, _, _ = shared_utils.load_statedict(disease_model, disease_trained_models[0], Configs)
disease_model = to_gpu(disease_model)

2022-07-24 09:59: Loading model - CheXpert/disease_prediction/trained_models/2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict


## Predictions

In [21]:
df_res_disease = pd.DataFrame(columns=Configs.DISEASE_ANNOTATIONS_COLUMNS + ['Mean'])
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean


In [22]:
cxp_disease_labels, cxp_disease_outputs = shared_utils.get_metric_tensors(disease_model, cxp_disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)
df_res_disease.loc['CXP'] = add_mean_to_list(shared_utils.auc_score(cxp_disease_labels, cxp_disease_outputs, per_class=True))

In [23]:
cxr_disease_labels, cxr_disease_outputs = shared_utils.get_metric_tensors(disease_model, cxr_disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)
df_res_disease.loc['CXR'] = add_mean_to_list(shared_utils.auc_score(cxr_disease_labels, cxr_disease_outputs, per_class=True))

In [24]:
df_res_disease.sort_values(by="Mean", ascending=False, inplace=True)
df_res_disease = df_res_disease.round(2)

In [25]:
df_res_disease

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
CXP,0.84,0.83,0.93,0.94,0.94,0.90
CXR,0.82,0.79,0.75,0.84,0.91,0.82


## Performance Per Protected Groups 

In [26]:
# race, age group, and gender to ChexPertDiseaseDatatset
cxp_demo_df = CheXpertRaceDataset.generate_race_dummies(pd.read_csv(os.path.join(Configs.CHEXPERT_DATA_DIR,
                                                                                 Configs.CHEXPERT_DEMO_FILENAME)),
                                                       'PRIMARY_RACE', Configs.CHEXPERT_RACE_DICT)
cxp_race_df = cxp_demo_df[['PATIENT', 'race'] + Configs.RACE_ANNOTATIONS_COLUMNS].drop_duplicates()
cxp_disease_df_labels = cxp_disease_valid_dataset.df_labels.merge(cxp_race_df, how='left',
                                                                                left_on='patient_id', right_on='PATIENT')
cxp_disease_df_labels.race.fillna('Other', inplace=True)
cxp_disease_df_labels['age'] = cxp_disease_df_labels.Age.apply(shared_utils.age_to_age_group)
cxp_disease_df_labels['gender'] = cxp_disease_df_labels.Sex
cxp_disease_df_labels.head(2)

,original_path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,img_path,patient_id,study,view,PATIENT,race,Asian,Black,Hispanic,White,age,gender
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64541,study1,view1_frontal.jpg,patient64541,White,0.0,0.0,0.0,1.0,70-90,Male
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,/home/student/MLH/debiasing-racial-effect-in-m...,patient64542,study1,view1_frontal.jpg,patient64542,White,0.0,0.0,0.0,1.0,70-90,Male


In [27]:
cxr_disease_df_labels = cxr_disease_valid_dataset.df_labels.copy()
cxr_disease_df_labels.gender.replace({"M": "Male", "F": "Female"}, inplace=True)
cxr_disease_df_labels.head(2)

,subject_id,study_id,split,dicom_id,ethnicity,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,folder_number,img_path
0,12572459,58979187,train,fb4d17cc-9f6e33ab-c17806ff-ce02e947-1bdeeeda,BLACK/AFRICAN AMERICAN,Black,40-70,Female,0.0,1.0,0.0,1.0,0.0,12,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...
1,16443212,50746382,train,7f0de7b4-b5c5a032-2525c3aa-3c0d13c6-f850ca36,BLACK/AFRICAN AMERICAN,Black,40-70,Female,0.0,0.0,0.0,0.0,0.0,16,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...


In [28]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['race'])

Only one class present in y_true. ROC AUC score is not defined in that case.


,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
race,,,,,,
Asian,0.80,0.85,0.89,1.00,0.92,0.8920
Black,1.00,1.00,1.00,0.90,1.00,0.9800
Hispanic,1.00,0.75,1.00,NaN,1.00,0.9375
Other,0.78,0.84,0.93,0.91,0.98,0.8880
White,0.85,0.82,0.94,0.94,0.92,0.8940


In [29]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['gender'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
gender,,,,,,
Female,0.81,0.81,0.90,0.93,0.96,0.882
Male,0.85,0.85,0.96,0.94,0.92,0.904


In [30]:
shared_utils.auc_per_protected_group(cxp_disease_df_labels, Mode.Disease, Configs, cxp_disease_labels,
                                     cxp_disease_outputs, protected_groups=['age'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
age,,,,,,
20-40,0.92,0.89,0.94,1.00,0.90,0.930
40-70,0.84,0.82,0.94,0.92,0.94,0.892
70-90,0.77,0.83,0.93,0.94,0.92,0.878


In [31]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['race'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
race,,,,,,
Black,0.78,0.75,0.78,0.83,0.98,0.824
Hispanic,0.86,0.83,0.57,0.84,0.90,0.800


In [32]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['gender'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
gender,,,,,,
Female,0.82,0.79,0.75,0.84,0.91,0.822


In [33]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['age'])

,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Mean
age,,,,,,
40-70,0.82,0.79,0.75,0.84,0.91,0.822


In [34]:
shared_utils.auc_per_protected_group(cxr_disease_df_labels, Mode.Disease, Configs, cxr_disease_labels,
                                     cxr_disease_outputs, protected_groups=['gender', 'age','race'])

Atelectasis  Cardiomegaly  Consolidation  Edema  Pleural Effusion   Mean
gender age   race                                                                              
Female 40-70 Black            0.78          0.75           0.78   0.83              0.98  0.824
             Hispanic         0.86          0.83           0.57   0.84              0.90  0.800

# Race Prediction

## Validation Dataloaders

In [14]:
cxp_race_valid_dataset = CheXpertRaceDataset(data_dir=Configs.CHEXPERT_DATA_DIR, demo_filename=Configs.CHEXPERT_DEMO_FILENAME, 
                                             labels_filename=Configs.CHEXPERT_VALID_LABELS_FILENAME, transform=valid_transform)
cxp_race_valid_dataset.df_labels = cxp_race_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxp_race_valid_dataloader = DataLoader(cxp_race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxp_race_valid_dataset)

172

In [15]:
# cxr_disease_valid_dataset = CXRDataset.download_dataset(10, cxr_mode.Race, Configs.CXR_DATA_DIR,
#                                                         Configs.CXR_VALID_LABELS_FILENAME, **Configs.CXR_FILENAMES,
#                                                         transform=valid_transform, target_transform=None)
cxr_race_valid_dataset = CXRDataset(Mode.Race, Configs.CXR_DATA_DIR, Configs.CXR_VALID_LABELS_FILENAME,
                                    transform=valid_transform)
cxr_race_valid_dataset.df_labels = cxr_race_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG]
cxr_race_valid_dataloader = DataLoader(cxr_race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(cxr_race_valid_dataset)

2400

## Pretrained Models 

In [16]:
_, _, files = next(os.walk(Configs.CHEXPERT_RACE_TRAINED_MODELS_DIR))
race_trained_models = [os.path.join(Configs.CHEXPERT_RACE_TRAINED_MODELS_DIR, file) for file in files]
race_model_versions = [p.split('__')[1] for p in race_trained_models]
len(race_trained_models)

8

In [17]:
models_dict = {}
for model_version, model_path in zip(race_model_versions, race_trained_models):
    model = densenet121()
    if "shallow" in model_version:
        shallow_denseblock = int(model_version.split('_')[2][10:])
        layer_offset = 3 + 2 * shallow_denseblock
        num_features = model.features[layer_offset].norm.num_features
        model = model.features[:layer_offset]
        classifier_module = nn.Sequential(
            nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),   
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=num_features, out_features=num_features, bias=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
        model.add_module('classifier', classifier_module)
    else:
        num_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, num_features, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
    model.eval()
    model, results, _, _ = shared_utils.load_statedict(model, model_path, Configs, device='cpu')
    models_dict[model_version] = model

2022-07-24 17:21: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-15_29__densenet121_race_denseblock1_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2805__validAUC-0.9178.dict
2022-07-24 17:21: Loading model - CheXpert/race_prediction/trained_models/2022_07_19-02_27__densenet121_race_denseblock2_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2811__validAUC-0.9094.dict
2022-07-24 17:21: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-10_11__densenet121_race_classifier_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.732__validAUC-0.6978.dict
2022-07-24 17:21: Loading model - CheXpert/race_prediction/trained_models/2022_07_21-13_21__densenet121_race_denseblock2_shallow__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.3791__validAUC-0.9075.dict
2022-07-24 17:21: Loading model - CheXpert/race_prediction/trained_models/2022_07_18-21_46__densenet121_race_denseblock3_freezed__epoch-9__iter-10266__batch_size-16__

## Predictions 

In [33]:
df_res_race = pd.DataFrame(columns=Configs.RACE_ANNOTATIONS_COLUMNS + ['Mean'])
df_res_race

,Asian,Black,Hispanic,White,Mean


In [34]:
for model_version, model in tqdm(models_dict.items()):
    model = to_gpu(model)
    cxp_race_labels, cxp_race_outputs = shared_utils.get_metric_tensors(model, cxp_race_valid_dataloader, Configs,
                                                                        apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                        by_study=False, challenge_ann_only=None)
    cxr_race_labels, cxr_race_outputs = shared_utils.get_metric_tensors(model, cxr_race_valid_dataloader, Configs,
                                                                        apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                        by_study=False, challenge_ann_only=None)
    df_res_race.loc[f"CXP_{model_version}"] = add_mean_to_list(shared_utils.auc_score(cxp_race_labels, cxp_race_outputs, per_class=True))
    df_res_race.loc[f"CXR_{model_version}"] = add_mean_to_list(shared_utils.auc_score(cxr_race_labels, cxr_race_outputs, per_class=True))
    model.cpu()

  0%|          | 0/8 [00:00<?, ?it/s]

In [35]:
df_res_race.sort_values(by="Mean", ascending=False, inplace=True)
df_res_race = df_res_race.round(2)

In [36]:
df_res_race

,Asian,Black,Hispanic,White,Mean
CXP_densenet121_race,0.90,0.93,0.96,0.88,0.92
CXP_densenet121_race_denseblock1_freezed,0.92,0.90,0.96,0.90,0.92
CXP_densenet121_race_denseblock2_freezed,0.90,0.87,0.97,0.89,0.91
CXP_densenet121_race_denseblock2_shallow,0.88,0.91,0.94,0.89,0.91
CXP_densenet121_race_denseblock3_freezed,0.90,0.81,0.96,0.88,0.89
CXP_densenet121_race_denseblock1_shallow,0.92,0.80,0.89,0.87,0.87
CXP_densenet121_race_denseblock4_freezed,0.89,0.78,0.94,0.84,0.86
CXR_densenet121_race,0.91,0.86,0.63,0.87,0.82
CXR_densenet121_race_denseblock1_freezed,0.90,0.86,0.59,0.88,0.81
CXR_densenet121_race_denseblock2_freezed,0.88,0.87,0.59,0.87,0.80


## Performance Per Protected Groups

In [18]:
# race, age group, and gender to ChexPertDiseaseDatatset
cxp_race_df_labels = cxp_race_valid_dataset.df_labels[['PATIENT', 'race', 'AGE_AT_CXR', 'GENDER'] + Configs.RACE_ANNOTATIONS_COLUMNS].drop_duplicates()
cxp_race_df_labels['age'] = cxp_race_df_labels.AGE_AT_CXR.apply(shared_utils.age_to_age_group)
cxp_race_df_labels['gender'] = cxp_race_df_labels.GENDER
cxp_race_df_labels.head(2)

,PATIENT,race,AGE_AT_CXR,GENDER,Asian,Black,Hispanic,White,age,gender
0,patient64627,Asian,29,Female,1,0,0,0,20-40,Female
1,patient64546,White,55,Male,0,0,0,1,40-70,Male


In [19]:
cxr_race_df_labels = cxr_race_valid_dataset.df_labels.copy()
cxr_race_df_labels.gender.replace({"M": "Male", "F": "Female"}, inplace=True)
cxr_race_df_labels.head(2)

,subject_id,study_id,split,dicom_id,ethnicity,race,age,gender,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,folder_number,img_path,Asian,Black,Hispanic,White
0,19859532,50453930,train,a5369d17-994f6cf1-dee57320-252febcc-6906f561,ASIAN,Asian,20-40,Female,NaN,NaN,NaN,NaN,NaN,19,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...,1,0,0,0
1,17719678,54809507,train,5d2b5624-28073582-14636856-8a3597b6-4cbc4bf3,ASIAN,Asian,20-40,Female,NaN,NaN,NaN,NaN,1.0,17,data/MIMIC-CXR-JPG/physionet.org/files/mimic-c...,1,0,0,0


In [20]:
model = models_dict['densenet121_race']
model = to_gpu(model)
cxp_race_labels, cxp_race_outputs = shared_utils.get_metric_tensors(model, cxp_race_valid_dataloader, Configs,
                                                                    apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                    by_study=False, challenge_ann_only=None)
cxr_race_labels, cxr_race_outputs = shared_utils.get_metric_tensors(model, cxr_race_valid_dataloader, Configs,
                                                                    apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                                    by_study=False, challenge_ann_only=None)

KeyboardInterrupt: 

In [ ]:
shared_utils.auc_per_protected_group(cxp_race_df_labels, Mode.Race, Configs, cxp_race_labels,
                                     cxp_race_outputs, protected_groups=['gender'])

In [ ]:
shared_utils.auc_per_protected_group(cxp_race_df_labels, Mode.Race, Configs, cxp_race_labels,
                                     cxp_race_outputs, protected_groups=['age'])

In [42]:
shared_utils.auc_per_protected_group(cxr_race_df_labels, Mode.Race, Configs, cxr_race_labels,
                                     cxr_race_outputs, protected_groups=['gender'])

,Asian,Black,Hispanic,White,Mean
gender,,,,,
Female,0.92,0.85,0.62,0.85,0.81
Male,0.89,0.87,0.64,0.88,0.82


In [43]:
shared_utils.auc_per_protected_group(cxr_race_df_labels, Mode.Race, Configs, cxr_race_labels,
                                     cxr_race_outputs, protected_groups=['age'])

,Asian,Black,Hispanic,White,Mean
age,,,,,
20-40,0.87,0.86,0.62,0.85,0.8000
40-70,0.95,0.88,0.68,0.89,0.8500
70-90,0.90,0.86,0.60,0.87,0.8075


In [ ]:
shared_utils.auc_per_protected_group(cxr_race_df_labels, Mode.Race, Configs, cxr_race_labels,
                                     cxr_race_outputs, protected_groups=['gender', 'age'])

# Grad-Cam

In [21]:
model = models_dict['densenet121_race']
model = to_gpu(model)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [32]:
imgs, _ = next(iter(cxp_race_valid_dataloader))
imgs = to_gpu(imgs)
pred = model(imgs)

In [33]:
img = imgs[0]
pred = pred[0]

In [35]:
pred.argmax(dim=1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [27]:
pred = model(img).argmax(dim=1)

In [ ]:
class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        
        # get the pretrained DenseNet201 network
        self.densenet = densenet201(pretrained=True)
        
        # disect the network to access its last convolutional layer
        self.features_conv = self.densenet.features
        
        # add the average global pool
        self.global_avg_pool = nn.AvgPool2d(kernel_size=7, stride=1)
        
        # get the classifier of the vgg19
        self.classifier = self.densenet.classifier
        
        # placeholder for the gradients
        self.gradients = None
    
    # hook for the gradients of the activations
    def activations_hook(self, grad):
        self.gradients = grad
        
    def forward(self, x):
        x = self.features_conv(x)
        
        # register the hook
        h = x.register_hook(self.activations_hook)
        
        # don't forget the pooling
        x = self.global_avg_pool(x)
        x = x.view((1, 1920))
        x = self.classifier(x)
        return x
    
    def get_activations_gradient(self):
        return self.gradients
    
    def get_activations(self, x):
        return self.features_conv(x)